In [1]:
# HPC
import numpy as np
import xarray as xr

# Plotting
import matplotlib.pyplot as plt

# Paths and stuff
import os
import sys

# For surfvel data in tif files
import rasterio
from rasterio.transform import from_origin

sys.path.append('/home/shreyas/pySICOPOLIS/src')
from pySICOPOLIS import *

In [2]:
modelPath = '/home/shreyas/update_to_develop_sicopolis/sicopolis_spinups/sico_out/'
dataPath = '/scratch2/shreyas/GrIS_paleo_data/'

ds_surfvel_model_40km = xr.open_dataset(modelPath + 'grl40_bm5_paleo17a_nudged_CT4_BH0_m21ka_pkp/grl40_bm5_paleo17a_nudged_CT4_BH0_m21ka_pkp0007.nc')
ds_surfvel_model_40km["vs"] = ds_surfvel_model_40km['vh_s'].copy()
ds_surfvel_model_40km["vx"] = ds_surfvel_model_40km['vx_s_g'].copy()
ds_surfvel_model_40km["vy"] = ds_surfvel_model_40km['vy_s_g'].copy()
ds_surfvel_model_40km["vs_uncert"] = ds_surfvel_model_40km["vs"].copy()*0.0 + 10.0
ds_surfvel_model_40km["vx_uncert"] = ds_surfvel_model_40km["vx"].copy()*0.0 + 10.0
ds_surfvel_model_40km["vy_uncert"] = ds_surfvel_model_40km["vy"].copy()*0.0 + 10.0
ds_surfvel_model_40km.to_netcdf('/scratch2/shreyas/GrIS_paleo_data/fake_surfvel_data_40kms.nc', mode='w')

In [3]:
xModel10 = np.arange(-72.,97.,1.0)*10
yModel10 = np.arange(-345.,-56.,1.0)*10
xModel16 = np.arange(-72.,97.,1.6)*10
yModel16 = np.arange(-345.,-56.,1.6)*10
xModel40 = np.arange(-72.,97.,4.0)*10
yModel40 = np.arange(-345.,-56.,4.0)*10

In [4]:
def read_field_from_tif(file_path):

    # Open the file
    with rasterio.open(file_path) as dataset:

        # # Print metadata information about the GeoTIFF file
        # print(dataset.meta)
        
        # Read the data from the first band (if the image has multiple bands)
        field = dataset.read(1)

        # SSG: Invert y-axis to match Ralf's surfvel data convention
        field = field[::-1]
        
        # # Accessing additional information
        # print(f"Width: {dataset.width}")
        # print(f"Height: {dataset.height}")
        # print(f"Number of Bands: {dataset.count}")

        return field

vs_data = read_field_from_tif("/scratch2/shreyas/GrIS_paleo_data/greenland_vel_mosaic250_vs_v1.tif")
vx_data = read_field_from_tif("/scratch2/shreyas/GrIS_paleo_data/greenland_vel_mosaic250_vx_v1.tif")
vx_uncert_data = read_field_from_tif("/scratch2/shreyas/GrIS_paleo_data/greenland_vel_mosaic250_ex_v1.tif")
vy_data = read_field_from_tif("/scratch2/shreyas/GrIS_paleo_data/greenland_vel_mosaic250_vy_v1.tif")
vy_uncert_data = read_field_from_tif("/scratch2/shreyas/GrIS_paleo_data/greenland_vel_mosaic250_ey_v1.tif")

ds = xr.open_dataset("/scratch2/shreyas/GrIS_paleo_data/greenland_vel_mosaic250_vs_v1.nc")
xs = ds["x"].data / 1000.0
ys = ds["y"].data / 1000.0

In [5]:
def frac_area(x, y, xx, yy):
    """
    Compute the fraction of the area of the small rectangle (xx, yy)
    within the large rectangle (x, y).

    Parameters:
        x, y:   Length-2 arrays for the large rectangle [x1, x2], [y1, y2]
        xx, yy: Length-2 arrays for the small rectangle

    Returns:
        Fraction of area of (xx, yy) within (x, y)
    """
    x = sorted(x)
    y = sorted(y)
    xx = sorted(xx)
    yy = sorted(yy)

    dx_inv = 1.0 / (x[1] - x[0])
    dy_inv = 1.0 / (y[1] - y[0])

    xxm = [(xx[0] - x[0]) * dx_inv, (xx[1] - x[0]) * dx_inv]
    yym = [(yy[0] - y[0]) * dy_inv, (yy[1] - y[0]) * dy_inv]

    if (xxm[1] - xxm[0] > 1) or (yym[1] - yym[0] > 1):
        raise ValueError("Small rectangle larger than large rectangle!")

    # Case 1: Fully inside
    if 0 <= xxm[0] and xxm[1] <= 1 and 0 <= yym[0] and yym[1] <= 1:
        return 1.0

    # Partial overlaps along one edge
    if 0 <= xxm[0] and xxm[1] <= 1:
        if yym[0] < 0 < yym[1]:
            return yym[1] / (yym[1] - yym[0])
        elif yym[0] < 1 < yym[1]:
            return (1 - yym[0]) / (yym[1] - yym[0])
        else:
            return 0.0

    if 0 <= yym[0] and yym[1] <= 1:
        if xxm[0] < 0 < xxm[1]:
            return xxm[1] / (xxm[1] - xxm[0])
        elif xxm[0] < 1 < xxm[1]:
            return (1 - xxm[0]) / (xxm[1] - xxm[0])
        else:
            return 0.0

    # Partial overlaps at corners
    denom = (xxm[1] - xxm[0]) * (yym[1] - yym[0])
    if denom == 0:
        return 0.0

    if xxm[0] < 0 < xxm[1] and yym[0] < 0 < yym[1]:
        return xxm[1] * yym[1] / denom
    if xxm[0] < 1 < xxm[1] and yym[0] < 0 < yym[1]:
        return (1 - xxm[0]) * yym[1] / denom
    if xxm[0] < 0 < xxm[1] and yym[0] < 1 < yym[1]:
        return xxm[1] * (1 - yym[0]) / denom
    if xxm[0] < 1 < xxm[1] and yym[0] < 1 < yym[1]:
        return (1 - xxm[0]) * (1 - yym[0]) / denom

    return 0.0

def downsample_field(xs, ys, field_data, x_sico, y_sico, dx_ms, dx):
    """
    Downsample a 2D field using area-weighted averaging from fine to coarse grid.

    Parameters:
        xs, ys         - 1D fine grid coordinates (x_ms, y_ms)
        field_data     - 2D array on fine grid, shape (len(ys), len(xs))
        x_sico, y_sico - 1D coarse grid coordinates
        dx_ms          - fine grid spacing
        dx             - coarse grid spacing

    Returns:
        field_interp   - 2D array on coarse grid (len(y_sico), len(x_sico))
    """
    field_interp = np.zeros((len(y_sico), len(x_sico)))
    sum_weight = np.zeros_like(field_interp)
    sum_weight_max = (dx / dx_ms) ** 2

    for i, x in enumerate(x_sico):

        for j, y in enumerate(y_sico):
            if x < xs[0] or x > xs[-1] or y < ys[0] or y > ys[-1]:
                continue

            x1 = x - 0.5 * (dx + dx_ms)
            x2 = x + 0.5 * (dx + dx_ms)
            y1 = y - 0.5 * (dx + dx_ms)
            y2 = y + 0.5 * (dx + dx_ms)

            ii1 = int(np.floor((x1 - xs[0]) / dx_ms))
            ii2 = int(np.ceil((x2 - xs[0]) / dx_ms))
            jj1 = int(np.floor((y1 - ys[0]) / dx_ms))
            jj2 = int(np.ceil((y2 - ys[0]) / dx_ms))

            for ii in range(ii1, ii2):
                for jj in range(jj1, jj2):
                    if 0 <= ii < len(xs) and 0 <= jj < len(ys):
                        val = field_data[jj, ii]
                        if not np.isnan(val):
                            weight = frac_area(
                                [x - 0.5 * dx, x + 0.5 * dx],
                                [y - 0.5 * dx, y + 0.5 * dx],
                                [xs[ii] - 0.5 * dx_ms, xs[ii] + 0.5 * dx_ms],
                                [ys[jj] - 0.5 * dx_ms, ys[jj] + 0.5 * dx_ms],
                            )
                            sum_weight[j, i] += weight
                            field_interp[j, i] += weight * val

            if sum_weight[j, i] > 0.5 * sum_weight_max:
                field_interp[j, i] /= sum_weight[j, i]
            else:
                field_interp[j, i] = 0.0

    return field_interp

vx_data[vx_data < -1.5e9] = np.nan
vy_data[vy_data < -1.5e9] = np.nan
vs_data[vs_data < -1.5e9] = np.nan
vx_uncert_data[vx_uncert_data < -1.5e9] = np.nan
vy_uncert_data[vy_uncert_data < -1.5e9] = np.nan

vs_data_interp = downsample_field(xs=xs, ys=ys, field_data=vs_data, x_sico=xModel40, y_sico=yModel40, dx_ms=0.25, dx=40.0)
vx_data_interp = downsample_field(xs=xs, ys=ys, field_data=vx_data, x_sico=xModel40, y_sico=yModel40, dx_ms=0.25, dx=40.0)
vy_data_interp = downsample_field(xs=xs, ys=ys, field_data=vy_data, x_sico=xModel40, y_sico=yModel40, dx_ms=0.25, dx=40.0)
vx_uncert_data_interp = downsample_field(xs=xs, ys=ys, field_data=vx_uncert_data, x_sico=xModel40, y_sico=yModel40, dx_ms=0.25, dx=40.0)
vy_uncert_data_interp = downsample_field(xs=xs, ys=ys, field_data=vy_uncert_data, x_sico=xModel40, y_sico=yModel40, dx_ms=0.25, dx=40.0)

vx_uncert_data_interp[vx_uncert_data_interp == 0.0] = -2.e9
vy_uncert_data_interp[vy_uncert_data_interp == 0.0] = -2.e9

In [6]:
ds_surfvel_data_40km_orig = xr.open_dataset(dataPath + 'surfvel_data_40kms_orig.nc')
vs_orig = ds_surfvel_data_40km_orig["vs"].data
vs_data_interp_calc = np.sqrt(vx_data_interp**2 + vy_data_interp**2)
np.nansum(np.abs((vs_orig-vs_data_interp)/vs_orig))/np.sum(vs_orig > 0), np.nansum(np.abs((vs_orig-vs_data_interp_calc)/vs_orig))/np.sum(vs_orig > 0)

/tmp/ipykernel_2575514/1181234617.py:4: RuntimeWarning: invalid value encountered in divide
  np.nansum(np.abs((vs_orig-vs_data_interp)/vs_orig))/np.sum(vs_orig > 0), np.nansum(np.abs((vs_orig-vs_data_interp_calc)/vs_orig))/np.sum(vs_orig > 0)


(2.119570653391888e-08, 0.158914788798674)

In [7]:
da_vs_data_orig = xr.DataArray(
        data = vs_orig,
        dims = ["y", "x"],
        coords = dict(
            y = yModel40,
            x = xModel40,
        ),  
        attrs = dict(description="vs in ma^(-1)"),
    )

da_vs_data_interp = xr.DataArray(
        data = vs_data_interp,
        dims = ["y", "x"],
        coords = dict(
            y = yModel40,
            x = xModel40,
        ),  
        attrs = dict(description="vs in ma^(-1)"),
    )

da_vx_data_interp = xr.DataArray(
        data = vx_data_interp,
        dims = ["y", "x"],
        coords = dict(
            y = yModel40,
            x = xModel40,
        ),  
        attrs = dict(description="vx in ma^(-1)"),
    )

da_vy_data_interp = xr.DataArray(
        data = vy_data_interp,
        dims = ["y", "x"],
        coords = dict(
            y = yModel40,
            x = xModel40,
        ),  
        attrs = dict(description="vy in ma^(-1)"),
    )

da_vx_uncert_data_interp = xr.DataArray(
        data = vx_uncert_data_interp,
        dims = ["y", "x"],
        coords = dict(
            y = yModel40,
            x = xModel40,
        ),  
        attrs = dict(description="vx_uncert in ma^(-1)"),
    )

da_vy_uncert_data_interp = xr.DataArray(
        data = vy_uncert_data_interp,
        dims = ["y", "x"],
        coords = dict(
            y = yModel40,
            x = xModel40,
        ),  
        attrs = dict(description="vy_uncert in ma^(-1)"),
    )

ds_surfvel_data_40km = ds_surfvel_data_40km_orig.copy()
ds_surfvel_data_40km["vs_orig"] = da_vs_data_orig
ds_surfvel_data_40km["vs"] = da_vs_data_interp
ds_surfvel_data_40km["vx"] = da_vx_data_interp
ds_surfvel_data_40km["vy"] = da_vy_data_interp

ds_surfvel_data_40km["vx_uncert_unbounded"] = da_vx_uncert_data_interp
ds_surfvel_data_40km["vx_uncert"] = ds_surfvel_data_40km["vx_uncert_unbounded"].copy()
ratio_x = np.abs(vx_data_interp/vx_uncert_data_interp)
ds_surfvel_data_40km["vx_uncert"].data[ratio_x > 20.0] = 0.05*np.abs(vx_data_interp[ratio_x > 20.0])

ds_surfvel_data_40km["vy_uncert_unbounded"] = da_vy_uncert_data_interp
ds_surfvel_data_40km["vy_uncert"] = ds_surfvel_data_40km["vy_uncert_unbounded"].copy()
ratio_y = np.abs(vy_data_interp/vy_uncert_data_interp)
ds_surfvel_data_40km["vy_uncert"].data[ratio_y > 20.0] = 0.05*np.abs(vy_data_interp[ratio_y > 20.0])

!rm /scratch2/shreyas/GrIS_paleo_data/surfvel_data_40kms.nc
ds_surfvel_data_40km.to_netcdf(dataPath + 'surfvel_data_40kms.nc')